In [4]:
import cloudscraper
from bs4 import BeautifulSoup as bs
import webbrowser
import time
import random
import json
import os
# 網頁 URL
url = "https://www.mobile01.com/topiclist.php?f=804"

# 使用 cloudscraper 來繞過 Cloudflare 的挑戰
headers = {
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"
}
scraper = cloudscraper.create_scraper()
res = scraper.get(url, headers=headers).text

# 使用 BeautifulSoup 解析 HTML
soup = bs(res, "lxml")
# 找到所有分頁按鈕並提取最後一個頁碼
pagination_elements = soup.select('li.l-pagination__page > a.c-pagination[data-page]')
if not pagination_elements:
    raise ValueError("未找到任何分頁按鈕，請檢查選擇器是否正確。")

# 提取最後一個分頁的 data-page 值
last_page_number = int(pagination_elements[-1]["data-page"])
print("最後一個頁碼是:", last_page_number)

最後一個頁碼是: 30


In [2]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import time

# 建立強化版 scraper
scraper = cloudscraper.create_scraper(
    browser={
        'browser': 'chrome',
        'platform': 'windows',
        'mobile': False
    }
)

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "zh-TW,zh;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Referer": "https://www.google.com"
}

# 目標頁面列表（可改 f 參數與頁數）
base_url = "https://www.mobile01.com/"
pages = [f"https://www.mobile01.com/topiclist.php?f=804&p={i}" for i in range(1, 31)]

article_links = []
skipped_links = []

# 抓取每頁的文章連結
for page in pages:
    res = scraper.get(page, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    links = soup.select('a.c-link.u-ellipsis[href*="topicdetail.php?f=804"]')
    for link in links:
        href = link.get("href")
        if href and "topicdetail.php" in href:
            href = href.split("#")[0]  # 去除 fragment
            full_url = base_url + href
            if full_url not in article_links:
                article_links.append(full_url)

print(f"共抓到 {len(article_links)} 筆文章連結")

# 抓取每篇文章的主文資訊
def extract_article_info(url):
    res = scraper.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    publish_block = soup.select_one(".l-articlePage__publish")
    if not publish_block:
        print(f"頁面缺少主文 publish 區塊：{url}")
        skipped_links.append(url)
        return None

    title = publish_block.select_one("h1.t2")
    time_el = publish_block.select_one(".l-navigation__item.is-dockingHide .o-fNotes")
    content = publish_block.select_one("[itemprop='articleBody']")
    tags = publish_block.select(".l-input__item a.o-hashtag")

    result = {
        "url": url,
        "title": title.text.strip() if title else "",
        "post_time": time_el.text.strip() if time_el else "",
        "content": content.text.strip() if content else "",
        "tags": ", ".join([tag.text.strip() for tag in tags]) if tags else ""
    }

    return result

# 執行爬取
results = []
for idx, link in enumerate(article_links):
    print(f"正在處理第 {idx+1} 筆：{link}")
    try:
        info = extract_article_info(link)
        if info:
            results.append(info)
    except Exception as e:
        print(f"[錯誤] {link} 發生異常：{e}")
    time.sleep(1)

# 匯出為 CSV
df = pd.DataFrame(results)
df.to_csv("mobile01_articles804.csv", index=False, encoding='utf-8-sig')
print("✅ 已成功輸出為 mobile01_articles.csv")

# 匯出錯誤連結
if skipped_links:
    with open("skipped_links.txt", "w", encoding="utf-8") as f:
        for link in skipped_links:
            f.write(link + "\n")
    print(f"⚠️ 有 {len(skipped_links)} 筆文章缺少主文，已記錄到 skipped_links.txt")
else:
    print("🎉 所有文章皆成功解析！")


共抓到 420 筆文章連結
正在處理第 1 筆：https://www.mobile01.com/topicdetail.php?f=804&t=7106034
正在處理第 2 筆：https://www.mobile01.com/topicdetail.php?f=804&t=7106034&p=1
正在處理第 3 筆：https://www.mobile01.com/topicdetail.php?f=804&t=6746417
正在處理第 4 筆：https://www.mobile01.com/topicdetail.php?f=804&t=6746417&p=37
正在處理第 5 筆：https://www.mobile01.com/topicdetail.php?f=804&t=7104361
正在處理第 6 筆：https://www.mobile01.com/topicdetail.php?f=804&t=7104361&p=1
正在處理第 7 筆：https://www.mobile01.com/topicdetail.php?f=804&t=7101944
正在處理第 8 筆：https://www.mobile01.com/topicdetail.php?f=804&t=7101944&p=1
正在處理第 9 筆：https://www.mobile01.com/topicdetail.php?f=804&t=7094547
正在處理第 10 筆：https://www.mobile01.com/topicdetail.php?f=804&t=7094547&p=1
正在處理第 11 筆：https://www.mobile01.com/topicdetail.php?f=804&t=6502139
正在處理第 12 筆：https://www.mobile01.com/topicdetail.php?f=804&t=6502139&p=1
正在處理第 13 筆：https://www.mobile01.com/topicdetail.php?f=804&t=7101848
正在處理第 14 筆：https://www.mobile01.com/topicdetail.php?f=804&t=7101848&p=8
正在處理第 15 筆：htt

In [1]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# 建立 scraper
scraper = cloudscraper.create_scraper(
    browser={'browser': 'chrome', 'platform': 'windows', 'mobile': False}
)

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "zh-TW,zh;q=0.9",
    "Referer": "https://www.google.com"
}

# ====== 抓文章樓層區塊 ======

def get_total_pages(url):
    res = scraper.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    links = soup.select("a.c-pagination")
    pages = [int(re.search(r'p=(\d+)', a.get('href', '')).group(1)) for a in links if re.search(r'p=(\d+)', a.get('href', ''))]
    return max(pages) if pages else 1

def extract_all_floors(url):
    all_floors = []
    total_pages = get_total_pages(url)

    for p in range(1, total_pages + 1):
        full_url = f"{url}&p={p}" if "?p=" not in url else url
        print(f"📄 抓取第 {p} 頁：{full_url}")
        try:
            res = scraper.get(full_url, headers=headers)
            soup = BeautifulSoup(res.text, "html.parser")

            title = soup.select_one("h1.t2")
            floor_blocks = soup.select('div[itemprop="articleBody"], article.c-articleLimit')
            time_tags = soup.select('span.o-fNotes.o-fSubMini')

            for i, block in enumerate(floor_blocks):
                content = block.get_text(strip=True)
                time_text = time_tags[i].text.strip() if i < len(time_tags) else ""
                all_floors.append({
                    "url": url,
                    "page": p,
                    "title": title.text.strip() if title else "",
                    "post_time": time_text,
                    "content": content
                })
        except Exception as e:
            print(f"❌ 發生錯誤於頁 {p}：{e}")
        time.sleep(1)
    return all_floors

# ====== 抓文章連結 ======

base_url = "https://www.mobile01.com/"
pages = [f"https://www.mobile01.com/topiclist.php?f=804&p={i}" for i in range(1, 31)]

article_links = []
skipped_links = []

for page in pages:
    res = scraper.get(page, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    links = soup.select('a.c-link.u-ellipsis[href*="topicdetail.php?f=804"]')
    for link in links:
        href = link.get("href").split("#")[0]
        full_url = base_url + href
        if full_url not in article_links:
            article_links.append(full_url)

print(f"✅ 共抓到 {len(article_links)} 筆文章連結")

# ====== 執行爬取每篇文章所有樓層 ======

all_results = []
for idx, link in enumerate(article_links):
    print(f"\n🔍 正在處理第 {idx+1} 筆：{link}")
    try:
        floors = extract_all_floors(link)
        if floors:
            all_results.extend(floors)
        else:
            skipped_links.append(link)
    except Exception as e:
        print(f"[錯誤] {link} 發生異常：{e}")
        skipped_links.append(link)

# ====== 輸出結果 ======

df = pd.DataFrame(all_results)
df.to_csv("mobile01_articles804_full.csv", index=False, encoding='utf-8-sig')
print(f"\n✅ 已成功輸出 {len(df)} 筆樓層資料至 mobile01_articles804_full.csv")

if skipped_links:
    with open("skipped_links_804.txt", "w", encoding="utf-8") as f:
        for link in skipped_links:
            f.write(link + "\n")
    print(f"⚠️ 有 {len(skipped_links)} 筆主文失敗，已記錄至 skipped_links_804.txt")
else:
    print("🎉 所有主文皆成功解析！")


✅ 共抓到 360 筆文章連結

🔍 正在處理第 1 筆：https://www.mobile01.com/topicdetail.php?f=804&t=7106034
📄 抓取第 1 頁：https://www.mobile01.com/topicdetail.php?f=804&t=7106034&p=1

🔍 正在處理第 2 筆：https://www.mobile01.com/topicdetail.php?f=804&t=7106034&p=1
📄 抓取第 1 頁：https://www.mobile01.com/topicdetail.php?f=804&t=7106034&p=1&p=1

🔍 正在處理第 3 筆：https://www.mobile01.com/topicdetail.php?f=804&t=6746417
📄 抓取第 1 頁：https://www.mobile01.com/topicdetail.php?f=804&t=6746417&p=1
📄 抓取第 2 頁：https://www.mobile01.com/topicdetail.php?f=804&t=6746417&p=2
📄 抓取第 3 頁：https://www.mobile01.com/topicdetail.php?f=804&t=6746417&p=3
📄 抓取第 4 頁：https://www.mobile01.com/topicdetail.php?f=804&t=6746417&p=4
📄 抓取第 5 頁：https://www.mobile01.com/topicdetail.php?f=804&t=6746417&p=5
📄 抓取第 6 頁：https://www.mobile01.com/topicdetail.php?f=804&t=6746417&p=6
📄 抓取第 7 頁：https://www.mobile01.com/topicdetail.php?f=804&t=6746417&p=7
📄 抓取第 8 頁：https://www.mobile01.com/topicdetail.php?f=804&t=6746417&p=8
📄 抓取第 9 頁：https://www.mobile01.com/topicdetail.php?f=804